# EMRI Example using FEW

To run this example you will have to have few installed.
This can done by installing this package with the dev option.

`pip install . [dev]`

In [32]:
import matplotlib.pyplot as plt

import numpy as np

from few.amplitude.romannet import RomanAmplitude
from few.trajectory.ode.flux import SchwarzEccFlux
from few.trajectory.inspiral import EMRIInspiral
from few.utils.ylm import GetYlms
from few.waveform import FastSchwarzschildEccentricFlux
from few.summation.interpolatedmodesum import CubicSplineInterpolant

traj = EMRIInspiral(func=SchwarzEccFlux)

In [29]:
inspiral_kwargs = {"DENSE_STEPPING": 0, "max_init_len": int(1e3)}
amplitude_kwargs = {}
Ylm_kwargs = {}
sum_kwargs = {"pad_output": False}

ylm_gen = GetYlms()

few = FastSchwarzschildEccentricFlux(inspiral_kwargs=inspiral_kwargs,
                                     amplitude_kwargs=amplitude_kwargs,
                                     Ylm_kwargs=Ylm_kwargs,
                                     sum_kwargs=sum_kwargs)



In [30]:
# parameters
M = 1e6
mu = 5e1
p0 = 10.0
e0 = 0.4
theta = np.pi/4.
phi = np.pi/3.

dist = 1
dt = 10.
T = 1.0

# evolve trajectory
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, 0.0, p0, e0, 1.0, T=T)

In [ ]:
# set the mode to compute
l_sel = 2
m_sel = 2
n_sel = 0
specific_modes = [(l_sel, m_sel, n_sel)]

amp = RomanAmplitude()

teuk_modes = amp(0., p, e, x, specific_modes=specific_modes)

ylms = ylm_gen(amp.unique_l, amp.unique_m, theta, phi).copy()[amp.inverse_lm]

In [34]:
# phase evolution of the harmonic
phase_evolution = m_sel*Phi_phi + n_sel*Phi_r

# spline the phase evolutions
interp_in = phase_evolution
spline = CubicSplineInterpolant(t, interp_in)
phase_spline = spline

t_new = np.linspace(t[0], t[-1], 1000)
new_phase_evolution = spline(t_new)